In [13]:
import cv2
import mediapipe as mp
import numpy as np
import time
import pyttsx3
mp_drawing = mp.solutions.drawing_utils #my notes=  to visulize the lines on the body 
mp_pose = mp.solutions.pose # from solutions we want the pose estemation model

In [14]:
pip install pyttsx3

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### calculate the angle 

In [15]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0]) #y2-y1 , x2-x1 , here we calculated the dgree in radians
    angle = np.abs(radians*180.0/np.pi) # we converted radiance to dgree
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

#### left shoulder abduction

In [16]:
cap = cv2.VideoCapture(0)

# shoulder abduction counter variables
counter = 0 
stage = None #down or up

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # mirror the webcam feed
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # Calculate angle
            angle = calculate_angle(left_hip, left_shoulder,left_elbow )
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # shoulder abduction counter logic
            if angle > 170:
                stage = "up"
            if angle < 30 and stage =='up':
                stage="down"
                counter +=1
                print(counter)
                       
        except:
            pass
        
        # Render shoulder abduction counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1) # coordinataet and color.-1 means fill the box
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

##### left shoulder abduction- ill just change how the reps are calculated

In [17]:
cap = cv2.VideoCapture(0)

# shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False


## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # mirror the webcam feed
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # Calculate angle
            angle = calculate_angle(left_hip, left_shoulder,left_elbow )
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # shoulder abduction counter logic
            if angle < 30:
                stage = "down"
                if rep_completed:
                   counter += 1
                   rep_completed = False
                   print(counter)
            elif angle > 170 and stage == "down":
                stage="up"
                rep_completed = True
                
                       
        except:
            pass
        
        # Render shoulder abduction counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1) # coordinataet and color.-1 means fill the box
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        #cv2.putText(image, 'STAGE', (65,12), 
                    #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        #cv2.putText(image, stage, 
                   # (60,60), 
                   # cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

##### left shoulder abduction- ill just change how the reps are calculated - the user will enter how many reps he want + motivitinal phrase at the end 

In [18]:
cap = cv2.VideoCapture(0)

# shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))


## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # mirror the webcam feed
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # Calculate angle
            angle = calculate_angle(left_hip, left_shoulder,left_elbow )
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # shoulder abduction counter logic
            if angle < 30:
                stage = "down"
                if rep_completed:
                   counter += 1
                   rep_completed = False
                   print(counter)

                   if counter >= target_reps:
                       print("Great job! You smashed your goal 💪")
                       break

            elif angle > 170 and stage == "down":
                stage="up"
                rep_completed = True
                        
        except:
            pass
        
        # Render shoulder abduction counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1) # coordinataet and color.-1 means fill the box
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        #cv2.putText(image, 'STAGE', (65,12), 
                    #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        #cv2.putText(image, stage, 
                   # (60,60), 
                   # cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


note: same code just the motivinal phrase will be printed on the screen

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time

cap = cv2.VideoCapture(0)

# shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))


## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # mirror the webcam feed
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # Calculate angle
            angle = calculate_angle(left_hip, left_shoulder,left_elbow )
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # shoulder abduction counter logic
            if angle < 30:
                stage = "down"
                if rep_completed:
                   counter += 1
                   rep_completed = False
                   print(counter)

                   if counter >= target_reps:
                       cv2.putText(image, "Great job! You smashed your goal", 
                                   (50, 200), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 
                                   1, (0, 0, 255), 3, cv2.LINE_AA)
                       cv2.imshow('Mediapipe Feed', image)
                       cv2.waitKey(1)
                       time.sleep(5)
                       break

            elif angle > 170 and stage == "down":
                stage="up"
                rep_completed = True
                        
        except:
            pass
        
        # Render shoulder abduction counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1) # coordinataet and color.-1 means fill the box
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        #cv2.putText(image, 'STAGE', (65,12), 
                    #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        #cv2.putText(image, stage, 
                   # (60,60), 
                   # cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1
2


with a voice

In [ ]:
cap = cv2.VideoCapture(0)

# shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Text-to-speech engine
engine = pyttsx3.init()

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # mirror the webcam feed
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # Calculate angle
            angle = calculate_angle(left_hip, left_shoulder,left_elbow )
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # shoulder abduction counter logic
            if angle < 30:
                stage = "down"
                if rep_completed:
                   counter += 1
                   rep_completed = False
                   print(counter)

                   if counter >= target_reps:
                       message = "Great job! You smashed your goal"
                       cv2.putText(image, message ,
                                   (50, 200), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 
                                   1, (0, 0, 255), 3, cv2.LINE_AA)
                       cv2.imshow('Mediapipe Feed', image)
                       cv2.waitKey(1)
                       engine.say(message)
                       engine.runAndWait()
                       break

            elif angle > 170 and stage == "down":
                stage="up"
                rep_completed = True
                        
        except:
            pass
        
        # Render shoulder abduction counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1) # coordinataet and color.-1 means fill the box
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        #cv2.putText(image, 'STAGE', (65,12), 
                    #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        #cv2.putText(image, stage, 
                   # (60,60), 
                   # cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


1
2


same code as above but with alert

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3

# Initialize mediapipe and text-to-speech
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# Helper function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

cap = cv2.VideoCapture(0)

# Shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate angles
            angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            
            # Visualize shoulder angle
            cv2.putText(image, str(int(angle)), 
                        tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Alert if elbow is not straight
            if elbow_angle < 160:
                alert_text = "Straighten your elbow"
                cv2.putText(image, alert_text,
                            (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1, (0, 0, 255), 3, cv2.LINE_AA)
                engine.say(alert_text)
                engine.runAndWait()

            # Shoulder abduction counter logic
            if angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(counter)

                    if counter >= target_reps:
                        message = "Great job! You smashed your goal"
                        cv2.putText(image, message,
                                    (50, 200), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 
                                    1, (0, 0, 255), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1)
                        engine.say(message)
                        engine.runAndWait()
                        break

            elif angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True
                        
        except:
            pass
        
        # Render shoulder abduction counter
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))              
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


fixing the warning 

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3

# Initialize mediapipe and text-to-speech
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# Helper function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

cap = cv2.VideoCapture(0)

# Shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False  # Track elbow alert status

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate angles
            angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            
            # Visualize shoulder angle
            cv2.putText(image, str(int(angle)), 
                        tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Dynamic elbow alert
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    alert_text = "Straighten your elbow"
                    cv2.putText(image, alert_text,
                                (50, 150),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                1, (0, 0, 255), 3, cv2.LINE_AA)
                    engine.say(alert_text)
                    engine.runAndWait()
                    elbow_warning_issued = True
                else:
                    cv2.putText(image, "Elbow bent",
                                (50, 150),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                elbow_warning_issued = False

            # Shoulder abduction counter logic
            if angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(counter)

                    if counter >= target_reps:
                        message = "Great job! You smashed your goal"
                        cv2.putText(image, message,
                                    (50, 200), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 
                                    1, (0, 0, 255), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1)
                        engine.say(message)
                        engine.runAndWait()
                        break

            elif angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True
                        
        except:
            pass
        
        # Render shoulder abduction counter
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))              
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading

# Initialize mediapipe and text-to-speech
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# Text-to-speech function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Helper function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Start video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Shoulder abduction counter variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False

# Ask user for number of reps
target_reps = int(input("Enter number of reps: "))

# Mediapipe pose instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        # Convert to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process pose
        results = pose.process(image)

        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Extract relevant joints
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate angles
            angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

            # Display shoulder angle
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(left_shoulder, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Elbow straightness check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    alert_text = "Straighten your elbow"
                    speak(alert_text)
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                elbow_warning_issued = False

            # Shoulder abduction rep counter
            if angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")

                    if counter >= target_reps:
                        message = "Great job! You smashed your goal"
                        speak(message)
                        cv2.putText(image, message, (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1000)
                        break

            elif angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

        except Exception as e:
            pass

        # Draw UI
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


Exception in thread Thread-5 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_14892\1526210682.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


Rep: 1


Exception in thread Thread-8 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_14892\1526210682.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-9 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    sel

Rep: 2


Exception in thread Thread-15 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_14892\1526210682.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-16 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    s

Rep: 3


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading

# Initialize MediaPipe and TTS
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# Speak function (non-blocking)
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Angle calculation helper
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180 else angle

# Webcam setup
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Rep counting variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False

# Ghost animation index
ghost_index = 0

# Ask user for target reps
target_reps = int(input("Enter number of reps: "))

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            # Key points
            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Angles
            shoulder_angle = calculate_angle(l_hip, l_shoulder, l_elbow)
            elbow_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)

            # Display angle on screen
            cv2.putText(image, str(int(shoulder_angle)),
                        tuple(np.multiply(l_shoulder, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Elbow form check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    speak("Straighten your elbow")
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                elbow_warning_issued = False

            # Rep logic
            if shoulder_angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")
                    if counter >= target_reps:
                        message = "Great job! You smashed your goal"
                        speak(message)
                        cv2.putText(image, message, (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(2000)
                        break
            elif shoulder_angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

            # === GHOST ARM (Animated Shadow Arm) ===
            shoulder_px = np.multiply(l_shoulder, [640, 480]).astype(int)

            # Animate ghost (from side to raised)
            t = (np.sin(ghost_index / 10 * np.pi) + 1) / 2  # 0 to 1
            ghost_index = (ghost_index + 1) % 20

            # Ghost positions relative to shoulder
            ghost_elbow_offset = np.array([-80 * t, -100 * t])
            ghost_wrist_offset = np.array([-140 * t, -180 * t])
            ghost_elbow = shoulder_px + ghost_elbow_offset.astype(int)
            ghost_wrist = shoulder_px + ghost_wrist_offset.astype(int)

            # Draw ghost arm
            cv2.line(image, tuple(shoulder_px), tuple(ghost_elbow), (100, 100, 255), 2)
            cv2.line(image, tuple(ghost_elbow), tuple(ghost_wrist), (100, 100, 255), 2)
            cv2.circle(image, tuple(shoulder_px), 5, (100, 100, 255), -1)
            cv2.circle(image, tuple(ghost_elbow), 5, (100, 100, 255), -1)
            cv2.circle(image, tuple(ghost_wrist), 5, (100, 100, 255), -1)

        except Exception as e:
            pass

        # UI - Rep Counter Box
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Show frame
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Cleanup
cap.release()
cv2.destroyAllWindows()


Rep: 1


Exception in thread Thread-56 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_14892\3145726015.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


Rep: 2


user frindly:

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading
import time
import math

# Initialize mediapipe and TTS
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# TTS function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Angle calculation helper
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Video feed setup
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Exercise variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False
target_reps = int(input("Enter number of reps: "))

# Ghost animation timer
start_time = time.time()

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Extract joints
            l_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            l_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            l_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            l_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

            left_hip = [l_hip.x, l_hip.y]
            left_shoulder = [l_shoulder.x, l_shoulder.y]
            left_elbow = [l_elbow.x, l_elbow.y]
            left_wrist = [l_wrist.x, l_wrist.y]

            # Calculate angles
            shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

            # Display shoulder angle
            cv2.putText(image, str(int(shoulder_angle)),
                        tuple(np.multiply(left_shoulder, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Elbow check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    speak("Straighten your elbow")
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                elbow_warning_issued = False

            # Repetition logic
            if shoulder_angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")
                    if counter >= target_reps:
                        speak("Great job! You smashed your goal")
                        cv2.putText(image, "Great job!", (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1500)
                        break
            elif shoulder_angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

            # Draw ghost guidance arm (from shoulder outward)
            shoulder_px = np.multiply(left_shoulder, [640, 480]).astype(int)

            # Animation timing
            t = ((time.time() - start_time) % 2) / 2  # cycles every 2 seconds
            angle_deg = 30 + 60 * t  # 30° to 90°
            angle_rad = math.radians(angle_deg)

            # Arm segment lengths (you can tweak these)
            upper_arm_len = 100
            lower_arm_len = 90

            ghost_elbow = [
                shoulder_px[0] - upper_arm_len * math.cos(angle_rad),
                shoulder_px[1] - upper_arm_len * math.sin(angle_rad)
            ]
            ghost_wrist = [
                ghost_elbow[0] - lower_arm_len * math.cos(angle_rad),
                ghost_elbow[1] - lower_arm_len * math.sin(angle_rad)
            ]

            # Draw ghost arm
            cv2.line(image, tuple(shoulder_px), tuple(np.int32(ghost_elbow)), (100, 255, 255), 2)
            cv2.line(image, tuple(np.int32(ghost_elbow)), tuple(np.int32(ghost_wrist)), (100, 255, 255), 2)
            cv2.circle(image, tuple(shoulder_px), 5, (100, 255, 255), -1)
            cv2.circle(image, tuple(np.int32(ghost_elbow)), 5, (100, 255, 255), -1)
            cv2.circle(image, tuple(np.int32(ghost_wrist)), 5, (100, 255, 255), -1)

        except Exception as e:
            pass

        # UI overlays
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        # Draw pose landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Rep: 1


Exception in thread Thread-134 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_14892\1454027587.py", line 18, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


Rep: 2


Exception in thread Thread-135 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_14892\1454027587.py", line 18, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-137 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
   

Rep: 3


trial 1 : im trying to make the ghost arem mimic the user movments

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading
import time
import math

# Initialize mediapipe and TTS
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# TTS function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Angle calculation helper
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Video feed setup
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Exercise variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False
target_reps = int(input("Enter number of reps: "))

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Track right arm (mirrored feed)
            r_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            r_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            r_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            r_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

            shoulder = [r_shoulder.x, r_shoulder.y]
            elbow = [r_elbow.x, r_elbow.y]
            wrist = [r_wrist.x, r_wrist.y]
            hip = [r_hip.x, r_hip.y]

            # Calculate angles
            shoulder_angle = calculate_angle(hip, shoulder, elbow)
            elbow_angle = calculate_angle(shoulder, elbow, wrist)

            # Display shoulder angle
            cv2.putText(image, str(int(shoulder_angle)),
                        tuple(np.multiply(shoulder, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Elbow check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    speak("Straighten your elbow")
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                elbow_warning_issued = False

            # Repetition logic
            if shoulder_angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")
                    if counter >= target_reps:
                        speak("Great job! You smashed your goal")
                        cv2.putText(image, "Great job!", (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1500)
                        break
            elif shoulder_angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

            # Draw ghost arm that mimics user motion
            shoulder_px = np.multiply(shoulder, [640, 480]).astype(int)
            elbow_px = np.multiply(elbow, [640, 480]).astype(int)
            wrist_px = np.multiply(wrist, [640, 480]).astype(int)

            overlay = image.copy()
            cv2.line(overlay, tuple(shoulder_px), tuple(elbow_px), (100, 255, 255), 2)
            cv2.line(overlay, tuple(elbow_px), tuple(wrist_px), (100, 255, 255), 2)
            cv2.circle(overlay, tuple(shoulder_px), 5, (100, 255, 255), -1)
            cv2.circle(overlay, tuple(elbow_px), 5, (100, 255, 255), -1)
            cv2.circle(overlay, tuple(wrist_px), 5, (100, 255, 255), -1)
            alpha = 0.4
            image = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)

        except Exception:
            pass

        # UI overlays
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        # Draw pose landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Exception in thread Thread-177 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_14892\879723951.py", line 18, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-178 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    

Rep: 1


Exception in thread Thread-180 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_14892\879723951.py", line 18, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-181 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    

Rep: 2


Exception in thread Thread-189 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_14892\879723951.py", line 18, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


trial2: in this code the max angle is good but we need to make the first line the user should follow a little lower 

In [4]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading
import time
import math

# Initialize mediapipe and TTS
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# TTS function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Angle calculation helper
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Video feed setup
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Exercise variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False
target_reps = int(input("Enter number of reps: "))

# Ghost animation timer
start_time = time.time()

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Track right arm (mirrored feed, so user sees it as their right)
            r_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            r_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            r_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            r_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

            shoulder = [r_shoulder.x, r_shoulder.y]
            elbow = [r_elbow.x, r_elbow.y]
            wrist = [r_wrist.x, r_wrist.y]
            hip = [r_hip.x, r_hip.y]

            # Calculate angles
            shoulder_angle = calculate_angle(hip, shoulder, elbow)
            elbow_angle = calculate_angle(shoulder, elbow, wrist)

            # Display shoulder angle
            cv2.putText(image, str(int(shoulder_angle)),
                        tuple(np.multiply(shoulder, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Elbow check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    speak("Straighten your elbow")
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                elbow_warning_issued = False

            # Repetition logic
            if shoulder_angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")
                    if counter >= target_reps:
                        speak("Great job! You smashed your goal")
                        cv2.putText(image, "Great job!", (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1500)
                        break
            elif shoulder_angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

            # Sync ghost arm with real-time movement
            shoulder_px = np.multiply(shoulder, [640, 480]).astype(int)

            # Calculate ghost arm position based on user’s angle
            # Updated to limit the maximum angle to 176°
            max_angle = 90
            angle_rad = math.radians(min(shoulder_angle, max_angle))  # This uses the shoulder angle but caps it to 176°

            # Arm segment lengths
            upper_arm_len = 90
            lower_arm_len = 90

            # Calculate ghost elbow and wrist position based on angles
            ghost_elbow = [
                shoulder_px[0] + upper_arm_len * math.cos(angle_rad),
                shoulder_px[1] - upper_arm_len * math.sin(angle_rad)
            ]
            ghost_wrist = [
                ghost_elbow[0] + lower_arm_len * math.cos(angle_rad),
                ghost_elbow[1] - lower_arm_len * math.sin(angle_rad)
            ]

            # Draw ghost arm
            cv2.line(image, tuple(shoulder_px), tuple(np.int32(ghost_elbow)), (100, 255, 255), 2)
            cv2.line(image, tuple(np.int32(ghost_elbow)), tuple(np.int32(ghost_wrist)), (100, 255, 255), 2)
            cv2.circle(image, tuple(shoulder_px), 5, (100, 255, 255), -1)
            cv2.circle(image, tuple(np.int32(ghost_elbow)), 5, (100, 255, 255), -1)
            cv2.circle(image, tuple(np.int32(ghost_wrist)), 5, (100, 255, 255), -1)

        except Exception:
            pass

        # UI overlays
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        # Draw pose landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Exception in thread Thread-17 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_17992\3998390017.py", line 18, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-18 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    s

Rep: 1


trial 3: final results

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading
import time
import math

# Initialize mediapipe and TTS
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Video feed setup
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Exercise variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False
target_reps = int(input("Enter number of reps: "))

# Ghost arm settings
start_angle = 30
max_angle = 90
animation_duration = 1.0  # seconds up/down
ghost_loops = target_reps  # how many up-down ghost loops before live tracking

def draw_ghost_arm(image, shoulder_px, angle_deg):
    angle_rad = math.radians(angle_deg)
    upper_arm_len = 90
    lower_arm_len = 90
    ghost_elbow = [
        shoulder_px[0] + upper_arm_len * math.cos(angle_rad),
        shoulder_px[1] - upper_arm_len * math.sin(angle_rad)
    ]
    ghost_wrist = [
        ghost_elbow[0] + lower_arm_len * math.cos(angle_rad),
        ghost_elbow[1] - lower_arm_len * math.sin(angle_rad)
    ]
    cv2.line(image, tuple(shoulder_px), tuple(np.int32(ghost_elbow)), (100, 255, 255), 2)
    cv2.line(image, tuple(np.int32(ghost_elbow)), tuple(np.int32(ghost_wrist)), (100, 255, 255), 2)
    cv2.circle(image, tuple(shoulder_px), 5, (100, 255, 255), -1)
    cv2.circle(image, tuple(np.int32(ghost_elbow)), 5, (100, 255, 255), -1)
    cv2.circle(image, tuple(np.int32(ghost_wrist)), 5, (100, 255, 255), -1)

# Ghost animation loop before live tracking
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    for i in range(ghost_loops):
        for direction in ["up", "down"]:
            start_time = time.time()
            while time.time() - start_time < animation_duration:
                ret, frame = cap.read()
                frame = cv2.flip(frame, 1)
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                try:
                    landmarks = results.pose_landmarks.landmark
                    r_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    shoulder = [r_shoulder.x, r_shoulder.y]
                    shoulder_px = np.multiply(shoulder, [640, 480]).astype(int)

                    progress = (time.time() - start_time) / animation_duration
                    if direction == "up":
                        ghost_angle = start_angle + (max_angle - start_angle) * progress
                    else:
                        ghost_angle = max_angle - (max_angle - start_angle) * progress

                    draw_ghost_arm(image, shoulder_px, ghost_angle)

                except Exception:
                    pass

                cv2.putText(image, f"Demo {i+1}/{ghost_loops}", (10, 470),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                cv2.imshow('Mediapipe Feed', image)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    exit()

    # Begin real-time exercise tracking
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            r_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            r_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            r_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            r_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

            shoulder = [r_shoulder.x, r_shoulder.y]
            elbow = [r_elbow.x, r_elbow.y]
            wrist = [r_wrist.x, r_wrist.y]
            hip = [r_hip.x, r_hip.y]

            shoulder_angle = calculate_angle(hip, shoulder, elbow)
            elbow_angle = calculate_angle(shoulder, elbow, wrist)

            cv2.putText(image, str(int(shoulder_angle)),
                        tuple(np.multiply(shoulder, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            if elbow_angle < 160:
                if not elbow_warning_issued:
                    speak("Straighten your elbow")
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                elbow_warning_issued = False

            if shoulder_angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")
                    if counter >= target_reps:
                        speak("Great job! You smashed your goal")
                        cv2.putText(image, "Great job!", (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(2000)
                        break
            elif shoulder_angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

            shoulder_px = np.multiply(shoulder, [640, 480]).astype(int)
            draw_ghost_arm(image, shoulder_px, min(shoulder_angle, max_angle))

        except Exception:
            pass

        # UI overlays
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Exception in thread Thread-5 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_19372\2043709499.py", line 17, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-6 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    sel

Rep: 1


Exception in thread Thread-9 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_19372\2043709499.py", line 17, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-10 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    se

Rep: 2


In [3]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading
import math

# Initialize MediaPipe and TTS
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

def draw_static_ghost_arm(image, shoulder_px, angle_deg, color=(100, 255, 255)):
    angle_rad = math.radians(angle_deg)
    upper_arm_len = 90
    lower_arm_len = 90
    ghost_elbow = [
        shoulder_px[0] + upper_arm_len * math.cos(angle_rad),
        shoulder_px[1] - upper_arm_len * math.sin(angle_rad)
    ]
    ghost_wrist = [
        ghost_elbow[0] + lower_arm_len * math.cos(angle_rad),
        ghost_elbow[1] - lower_arm_len * math.sin(angle_rad)
    ]
    cv2.line(image, tuple(shoulder_px), tuple(np.int32(ghost_elbow)), color, 2)
    cv2.line(image, tuple(np.int32(ghost_elbow)), tuple(np.int32(ghost_wrist)), color, 2)
    cv2.circle(image, tuple(shoulder_px), 5, color, -1)
    cv2.circle(image, tuple(np.int32(ghost_elbow)), 5, color, -1)
    cv2.circle(image, tuple(np.int32(ghost_wrist)), 5, color, -1)

# Video feed setup
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Exercise variables
counter = 0
stage = "start"
target_reps = int(input("Enter number of reps: "))

# Static angles sequence (rep path)
rep_sequence = [70,90]  # Degrees
current_target_idx = 0

def angle_close(a, b, threshold=10):
    return abs(a - b) <= threshold

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            r_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            r_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            r_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            r_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

            shoulder = [r_shoulder.x, r_shoulder.y]
            elbow = [r_elbow.x, r_elbow.y]
            wrist = [r_wrist.x, r_wrist.y]
            hip = [r_hip.x, r_hip.y]

            shoulder_angle = calculate_angle(hip, shoulder, elbow)

            shoulder_px = np.multiply(shoulder, [640, 480]).astype(int)

            # Draw the current ghost arm
            target_angle = rep_sequence[current_target_idx]
            draw_static_ghost_arm(image, shoulder_px, target_angle)

            # Text feedback
            cv2.putText(image, f"Target: {target_angle} deg", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            cv2.putText(image, f"Your Angle: {int(shoulder_angle)}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            if angle_close(shoulder_angle, target_angle):
                current_target_idx += 1
                if current_target_idx >= len(rep_sequence):
                    counter += 1
                    speak(f"Good job. Repetition {counter} complete")
                    current_target_idx = 0

            if counter >= target_reps:
                speak("Exercise complete. Well done!")
                cv2.putText(image, "Complete!", (200, 240),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
                cv2.imshow('Mediapipe Feed', image)
                cv2.waitKey(3000)
                break

        except Exception:
            pass

        # Counter UI
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Exception in thread Thread-13 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-14 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipyker

another method

In [21]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading
import time
import math

# Initialize mediapipe and TTS
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# TTS function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Angle calculation helper
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Video feed setup
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Exercise variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False
target_reps = int(input("Enter number of reps: "))

# Ghost angles
guidance_angles = [-90,0, 90]
upper_arm_len = 90
lower_arm_len = 90

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Track right arm (mirrored feed)
            r_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            r_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            r_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            r_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

            shoulder = [r_shoulder.x, r_shoulder.y]
            elbow = [r_elbow.x, r_elbow.y]
            wrist = [r_wrist.x, r_wrist.y]
            hip = [r_hip.x, r_hip.y]

            shoulder_px = np.multiply(shoulder, [640, 480]).astype(int)

            # Calculate angles
            shoulder_angle = calculate_angle(hip, shoulder, elbow)
            elbow_angle = calculate_angle(shoulder, elbow, wrist)

            # Display shoulder angle
            cv2.putText(image, str(int(shoulder_angle)),
                        tuple(shoulder_px),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Elbow check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    speak("Straighten your elbow")
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                elbow_warning_issued = False

            # Repetition logic
            if shoulder_angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")
                    if counter >= target_reps:
                        speak("Great job! You smashed your goal")
                        cv2.putText(image, "Great job!", (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1500)
                        break
            elif shoulder_angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

            # Draw ghost arms from user's actual shoulder
            for angle in guidance_angles:
                angle_rad = math.radians(angle)
                ghost_elbow = [
                    shoulder_px[0] + upper_arm_len * math.cos(angle_rad),
                    shoulder_px[1] - upper_arm_len * math.sin(angle_rad)
                ]
                ghost_wrist = [
                    ghost_elbow[0] + lower_arm_len * math.cos(angle_rad),
                    ghost_elbow[1] - lower_arm_len * math.sin(angle_rad)
                ]

                cv2.line(image, tuple(shoulder_px), tuple(np.int32(ghost_elbow)), (0, 255, 0), 2)
                cv2.line(image, tuple(np.int32(ghost_elbow)), tuple(np.int32(ghost_wrist)), (0, 255, 0), 2)
                cv2.circle(image, tuple(shoulder_px), 4, (0, 255, 0), -1)
                cv2.circle(image, tuple(np.int32(ghost_elbow)), 4, (0, 255, 0), -1)
                cv2.circle(image, tuple(np.int32(ghost_wrist)), 4, (0, 255, 0), -1)

                # Optional: Label the angle
                cv2.putText(image, f"{angle}°",
                            tuple(np.int32(ghost_elbow)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        except Exception:
            pass

        # UI overlays
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        # Draw pose landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Exception in thread Thread-241 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_17992\2161624394.py", line 18, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-243 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
   

Rep: 1


Exception in thread Thread-244 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_17992\2161624394.py", line 18, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


Rep: 2


Exception in thread Thread-247 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_17992\2161624394.py", line 18, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


Rep: 3


Exception in thread Thread-248 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_17992\2161624394.py", line 18, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-250 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
   

Rep: 4


Exception in thread Thread-253 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_17992\2161624394.py", line 18, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-254 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
   

Rep: 5


i will make it with no angle + enchane the color 

In [15]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading
import time
import math

# Initialize mediapipe and TTS
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# TTS function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Angle calculation helper
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Video feed setup
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Exercise variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False
target_reps = int(input("Enter number of reps: "))

# Ghost angles
guidance_angles = [-90, 0, 90]
upper_arm_len = 90
lower_arm_len = 90

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Track right arm (mirrored feed)
            r_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            r_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            r_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            r_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

            shoulder = [r_shoulder.x, r_shoulder.y]
            elbow = [r_elbow.x, r_elbow.y]
            wrist = [r_wrist.x, r_wrist.y]
            hip = [r_hip.x, r_hip.y]

            shoulder_px = np.multiply(shoulder, [640, 480]).astype(int)

            # Calculate angles
            shoulder_angle = calculate_angle(hip, shoulder, elbow)
            elbow_angle = calculate_angle(shoulder, elbow, wrist)

            # Elbow check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    speak("Straighten your elbow")
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                elbow_warning_issued = False

            # Repetition logic
            if shoulder_angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")
                    if counter >= target_reps:
                        speak("Great job! You smashed your goal")
                        cv2.putText(image, "Great job!", (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1500)
                        break
            elif shoulder_angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

            # Draw ghost arms from user's actual shoulder with blue color
            ghost_color = (255, 0, 0)  # RGB for blue

            for angle in guidance_angles:
                angle_rad = math.radians(angle)
                ghost_elbow = [
                    shoulder_px[0] + upper_arm_len * math.cos(angle_rad),
                    shoulder_px[1] - upper_arm_len * math.sin(angle_rad)
                ]
                ghost_wrist = [
                    ghost_elbow[0] + lower_arm_len * math.cos(angle_rad),
                    ghost_elbow[1] - lower_arm_len * math.sin(angle_rad)
                ]

                cv2.line(image, tuple(shoulder_px), tuple(np.int32(ghost_elbow)), ghost_color, 2)
                cv2.line(image, tuple(np.int32(ghost_elbow)), tuple(np.int32(ghost_wrist)), ghost_color, 2)
                cv2.circle(image, tuple(shoulder_px), 4, ghost_color, -1)
                cv2.circle(image, tuple(np.int32(ghost_elbow)), 4, ghost_color, -1)
                cv2.circle(image, tuple(np.int32(ghost_wrist)), 4, ghost_color, -1)

        except Exception:
            pass

        # UI overlays
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        # Draw pose landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Exception in thread Thread-272 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_19100\2595303460.py", line 18, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-274 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
   

i will make the lines dispear

In [5]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading
import time
import math

# Initialize mediapipe and TTS
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

# TTS function using threading
def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Angle calculation helper
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Video feed setup
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Exercise variables
counter = 0
stage = "down"
rep_completed = False
elbow_warning_issued = False
target_reps = int(input("Enter number of reps: "))

# Ghost arm logic
ghost_sequence = [-90, 5, 90, 5, -90]
current_ghost_index = 0
ghost_tolerance = 15  # degrees
upper_arm_len = 90
lower_arm_len = 90

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Track right arm (mirrored feed)
            r_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            r_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            r_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            r_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

            shoulder = [r_shoulder.x, r_shoulder.y]
            elbow = [r_elbow.x, r_elbow.y]
            wrist = [r_wrist.x, r_wrist.y]
            hip = [r_hip.x, r_hip.y]

            shoulder_px = np.multiply(shoulder, [640, 480]).astype(int)

            # Calculate angles
            shoulder_angle = calculate_angle(hip, shoulder, elbow)
            elbow_angle = calculate_angle(shoulder, elbow, wrist)

            # Elbow check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    speak("Straighten your elbow")
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                elbow_warning_issued = False

            # Repetition logic
            if shoulder_angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    print(f"Rep: {counter}")
                    if counter >= target_reps:
                        speak("Great job! You smashed your goal")
                        cv2.putText(image, "Great job!", (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(1500)
                        break
            elif shoulder_angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

            # Show only the current ghost arm
            ghost_color = (255, 0, 0)
            target_angle = ghost_sequence[current_ghost_index]

            angle_rad = math.radians(target_angle)
            ghost_elbow = [
                shoulder_px[0] + upper_arm_len * math.cos(angle_rad),
                shoulder_px[1] - upper_arm_len * math.sin(angle_rad)
            ]
            ghost_wrist = [
                ghost_elbow[0] + lower_arm_len * math.cos(angle_rad),
                ghost_elbow[1] - lower_arm_len * math.sin(angle_rad)
            ]

            cv2.line(image, tuple(shoulder_px), tuple(np.int32(ghost_elbow)), ghost_color, 2)
            cv2.line(image, tuple(np.int32(ghost_elbow)), tuple(np.int32(ghost_wrist)), ghost_color, 2)
            cv2.circle(image, tuple(shoulder_px), 4, ghost_color, -1)
            cv2.circle(image, tuple(np.int32(ghost_elbow)), 4, ghost_color, -1)
            cv2.circle(image, tuple(np.int32(ghost_wrist)), 4, ghost_color, -1)

            # Advance to next ghost if user reaches the target angle
            if abs(shoulder_angle - abs(target_angle)) < ghost_tolerance:
                current_ghost_index = (current_ghost_index + 1) % len(ghost_sequence)
                time.sleep(0.5)

        except Exception:
            pass

        # UI overlays
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        # Draw pose landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Exception in thread Thread-33 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_20340\4176714816.py", line 18, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-35 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    s

Rep: 1


Exception in thread Thread-37 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_20340\4176714816.py", line 18, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-39 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    s

Rep: 2


In [10]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading
import time
import math

# Initialize mediapipe and TTS
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return angle if angle <= 180.0 else 360 - angle

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Ghost logic variables
buffer = 15
upper_arm_len = 90
lower_arm_len = 90
ghost_sequence = [-90, 5, 90]
current_ghost_index = 0

def get_ghost_angles(current_step):
    if current_step == -90:
        return [5, 90]
    elif current_step == 5:
        return [90]
    elif current_step == 90:
        return [-90,5]
    return []

def check_transition(shoulder_angle):
    global current_ghost_index
    target_angles = get_ghost_angles(ghost_sequence[current_ghost_index])
    for angle in target_angles:
        if abs(shoulder_angle - abs(angle)) <= buffer:
            if current_ghost_index < len(ghost_sequence) - 1:
                current_ghost_index += 1
            else:
                current_ghost_index = 0
            break

# Rep counter
counter = 0
target_reps = int(input("Enter number of reps: "))
rep_completed = False
stage = "down"
elbow_warning_issued = False

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            r_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            r_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            r_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            r_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

            shoulder = [r_shoulder.x, r_shoulder.y]
            elbow = [r_elbow.x, r_elbow.y]
            wrist = [r_wrist.x, r_wrist.y]
            hip = [r_hip.x, r_hip.y]
            shoulder_px = np.multiply(shoulder, [640, 480]).astype(int)

            shoulder_angle = calculate_angle(hip, shoulder, elbow)
            elbow_angle = calculate_angle(shoulder, elbow, wrist)

            # Elbow straightness check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    speak("Straighten your elbow")
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                elbow_warning_issued = False

            # Repetition logic (optional logic; adjust as needed)
            if shoulder_angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    speak(f"Rep {counter}")
                    print(f"Rep: {counter}")
                    if counter >= target_reps:
                        speak("Great job! You smashed your goal.")
                        cv2.putText(image, "Great job!", (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(2000)
                        break
            elif shoulder_angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

            # Ghost transition logic
            check_transition(shoulder_angle)
            ghost_angles = get_ghost_angles(ghost_sequence[current_ghost_index])
            ghost_color = (255, 0, 0)

            for angle in ghost_angles:
                angle_rad = math.radians(angle)
                ghost_elbow = [
                    shoulder_px[0] + upper_arm_len * math.cos(angle_rad),
                    shoulder_px[1] - upper_arm_len * math.sin(angle_rad)
                ]
                ghost_wrist = [
                    ghost_elbow[0] + lower_arm_len * math.cos(angle_rad),
                    ghost_elbow[1] - lower_arm_len * math.sin(angle_rad)
                ]
                cv2.line(image, tuple(shoulder_px), tuple(np.int32(ghost_elbow)), ghost_color, 2)
                cv2.line(image, tuple(np.int32(ghost_elbow)), tuple(np.int32(ghost_wrist)), ghost_color, 2)
                cv2.circle(image, tuple(shoulder_px), 4, ghost_color, -1)
                cv2.circle(image, tuple(np.int32(ghost_elbow)), 4, ghost_color, -1)
                cv2.circle(image, tuple(np.int32(ghost_wrist)), 4, ghost_color, -1)

        except Exception:
            pass

        # UI overlay
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, str(counter), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Exception in thread Thread-68 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_20340\2399103031.py", line 17, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-70 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    s

best trial so far

In [14]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading
import math

# Initialize mediapipe and TTS
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return angle if angle <= 180.0 else 360 - angle

cap = cv2.VideoCapture(0)

# Ghost logic variables
buffer = 15
upper_arm_len = 90
lower_arm_len = 90
ghost_sequence = [-90, 5, 90]
current_ghost_index = 0

def get_ghost_angles(current_step):
    if current_step == -90:
        return [5, 90]
    elif current_step == 5:
        return [90]
    elif current_step == 90:
        return [-90, 5]
    return []

def check_transition(shoulder_angle):
    global current_ghost_index
    target_angle = ghost_sequence[current_ghost_index]
    if abs(shoulder_angle - abs(target_angle)) <= buffer:
        current_ghost_index = (current_ghost_index + 1) % len(ghost_sequence)

# Rep counter
counter = 0
target_reps = int(input("Enter number of reps: "))
rep_completed = False
stage = "down"
elbow_warning_issued = False

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            shoulder_px = np.multiply(shoulder, [640, 480]).astype(int)
            shoulder_angle = calculate_angle(hip, shoulder, elbow)
            elbow_angle = calculate_angle(shoulder, elbow, wrist)

            # Elbow straightness check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    speak("Straighten your elbow")
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                elbow_warning_issued = False

            # Repetition logic
            if shoulder_angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    speak(f"Rep {counter}")
                    print(f"Rep: {counter}")
                    if counter >= target_reps:
                        speak("Great job! You smashed your goal.")
                        cv2.putText(image, "Great job!", (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(2000)
                        break
            elif shoulder_angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

            # Ghost logic
            check_transition(shoulder_angle)
            ghost_angles = get_ghost_angles(ghost_sequence[current_ghost_index])

            for angle in ghost_angles:
                angle_rad = math.radians(angle)
                ghost_elbow = [
                    shoulder_px[0] + upper_arm_len * math.cos(angle_rad),
                    shoulder_px[1] - upper_arm_len * math.sin(angle_rad)
                ]
                ghost_wrist = [
                    ghost_elbow[0] + lower_arm_len * math.cos(angle_rad),
                    ghost_elbow[1] - lower_arm_len * math.sin(angle_rad)
                ]
                cv2.line(image, tuple(shoulder_px), tuple(np.int32(ghost_elbow)), (255, 0, 0), 2)
                cv2.line(image, tuple(np.int32(ghost_elbow)), tuple(np.int32(ghost_wrist)), (255, 0, 0), 2)
                cv2.circle(image, tuple(shoulder_px), 4, (255, 0, 0), -1)
                cv2.circle(image, tuple(np.int32(ghost_elbow)), 4, (255, 0, 0), -1)
                cv2.circle(image, tuple(np.int32(ghost_wrist)), 4, (255, 0, 0), -1)

        except Exception:
            pass

        # UI overlay
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, str(counter), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Exception in thread Thread-259 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_19100\3234980049.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-260 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
   

Rep: 1


Exception in thread Thread-266 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_19100\3234980049.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-267 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
   

Rep: 2


Exception in thread Thread-269 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_19100\3234980049.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-270 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
   

Rep: 3


In [6]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading
import math

# Initialize mediapipe and TTS
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return angle if angle <= 180.0 else 360 - angle

cap = cv2.VideoCapture(0)

# Ghost logic variables
buffer = 15
upper_arm_len = 90
lower_arm_len = 90
ghost_sequence = [-90, 5, 90]
current_ghost_index = 0

def get_ghost_angles(current_step, shoulder_angle):
    if abs(shoulder_angle - 90) <= buffer:
        return [-90, 5, 90]
    elif current_step == -90:
        return [5, 90]
    elif current_step == 5:
        return [90]
    elif current_step == 90:
        return [-90, 5]
    return []

def check_transition(shoulder_angle):
    global current_ghost_index
    target_angle = ghost_sequence[current_ghost_index]
    if abs(shoulder_angle - abs(target_angle)) <= buffer:
        current_ghost_index = (current_ghost_index + 1) % len(ghost_sequence)

# Rep counter
counter = 0
target_reps = int(input("Enter number of reps: "))
rep_completed = False
stage = "down"
elbow_warning_issued = False

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            shoulder_px = np.multiply(shoulder, [640, 480]).astype(int)
            shoulder_angle = calculate_angle(hip, shoulder, elbow)
            elbow_angle = calculate_angle(shoulder, elbow, wrist)

            # Elbow straightness check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    speak("Straighten your elbow")
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                elbow_warning_issued = False

            # Repetition logic
            if shoulder_angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    speak(f"Rep {counter}")
                    print(f"Rep: {counter}")
                    if counter >= target_reps:
                        speak("Great job! You smashed your goal.")
                        cv2.putText(image, "Great job!", (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(2000)
                        break
            elif shoulder_angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

            # Ghost logic
            check_transition(shoulder_angle)
            ghost_angles = get_ghost_angles(ghost_sequence[current_ghost_index], shoulder_angle)

            for angle in ghost_angles:
                angle_rad = math.radians(angle)
                ghost_elbow = [
                    shoulder_px[0] + upper_arm_len * math.cos(angle_rad),
                    shoulder_px[1] - upper_arm_len * math.sin(angle_rad)
                ]
                ghost_wrist = [
                    ghost_elbow[0] + lower_arm_len * math.cos(angle_rad),
                    ghost_elbow[1] - lower_arm_len * math.sin(angle_rad)
                ]
                cv2.line(image, tuple(shoulder_px), tuple(np.int32(ghost_elbow)), (255, 0, 0), 2)
                cv2.line(image, tuple(np.int32(ghost_elbow)), tuple(np.int32(ghost_wrist)), (255, 0, 0), 2)
                cv2.circle(image, tuple(shoulder_px), 4, (255, 0, 0), -1)
                cv2.circle(image, tuple(np.int32(ghost_elbow)), 4, (255, 0, 0), -1)
                cv2.circle(image, tuple(np.int32(ghost_wrist)), 4, (255, 0, 0), -1)

        except Exception:
            pass

        # UI overlay
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, str(counter), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Exception in thread Thread-94 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_19100\2532691703.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-95 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    s

Rep: 1


Exception in thread Thread-101 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_19100\2532691703.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-104 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
   

Rep: 2


Exception in thread Thread-106 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_19100\2532691703.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-107 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
   

...

Exception in thread Thread-110 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_19100\2532691703.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-115 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
   

Rep: 4


Exception in thread Thread-120 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_19100\2532691703.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-121 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
   

Rep: 5


Exception in thread Thread-126 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_19100\2532691703.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


i will try this code 

In [23]:
import cv2
import numpy as np
import mediapipe as mp
import pyttsx3
import threading
import math

# Initialize mediapipe and TTS
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
engine = pyttsx3.init()

def speak(text):
    def _speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return angle if angle <= 180.0 else 360 - angle

cap = cv2.VideoCapture(0)

# Ghost logic variables
buffer = 15
upper_arm_len = 90
lower_arm_len = 90
ghost_sequence = [-90, 5, 90]  # Sequence of target angles
current_ghost_index = 0  # Start at -90°

def get_ghost_angles(current_step):
    # Define the ghost arm sequence based on the current step
    if current_step == -90:
        return [5, 90]  # Show both 5° and 90° at -90°
    elif current_step == 5:
        return [90]  # Show only 90° at 5°
    elif current_step == 90:
        return [-90, 5]  # Show -90° and 5° at 90°
    return []

def check_transition(shoulder_angle):
    global current_ghost_index
    target_angle = ghost_sequence[current_ghost_index]
    if abs(shoulder_angle - abs(target_angle)) <= buffer:
        # Move to the next angle in the sequence
        current_ghost_index = (current_ghost_index + 1) % len(ghost_sequence)

# Rep counter
counter = 0
target_reps = int(input("Enter number of reps: "))
rep_completed = False
stage = "down"
elbow_warning_issued = False

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            shoulder_px = np.multiply(shoulder, [640, 480]).astype(int)
            shoulder_angle = calculate_angle(hip, shoulder, elbow)
            elbow_angle = calculate_angle(shoulder, elbow, wrist)

            # Elbow straightness check
            if elbow_angle < 160:
                if not elbow_warning_issued:
                    speak("Straighten your elbow")
                    elbow_warning_issued = True
                cv2.putText(image, "Elbow bent", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                elbow_warning_issued = False

            # Repetition logic
            if shoulder_angle < 30:
                stage = "down"
                if rep_completed:
                    counter += 1
                    rep_completed = False
                    speak(f"Rep {counter}")
                    print(f"Rep: {counter}")
                    if counter >= target_reps:
                        speak("Great job! You smashed your goal.")
                        cv2.putText(image, "Great job!", (50, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        cv2.imshow('Mediapipe Feed', image)
                        cv2.waitKey(2000)
                        break
            elif shoulder_angle > 170 and stage == "down":
                stage = "up"
                rep_completed = True

            # Ghost logic
            check_transition(shoulder_angle)
            ghost_angles = get_ghost_angles(ghost_sequence[current_ghost_index])

            for angle in ghost_angles:
                angle_rad = math.radians(angle)
                ghost_elbow = [
                    shoulder_px[0] + upper_arm_len * math.cos(angle_rad),
                    shoulder_px[1] - upper_arm_len * math.sin(angle_rad)
                ]
                ghost_wrist = [
                    ghost_elbow[0] + lower_arm_len * math.cos(angle_rad),
                    ghost_elbow[1] - lower_arm_len * math.sin(angle_rad)
                ]
                cv2.line(image, tuple(shoulder_px), tuple(np.int32(ghost_elbow)), (255, 0, 0), 2)
                cv2.line(image, tuple(np.int32(ghost_elbow)), tuple(np.int32(ghost_wrist)), (255, 0, 0), 2)
                cv2.circle(image, tuple(shoulder_px), 4, (255, 0, 0), -1)
                cv2.circle(image, tuple(np.int32(ghost_elbow)), 4, (255, 0, 0), -1)
                cv2.circle(image, tuple(np.int32(ghost_wrist)), 4, (255, 0, 0), -1)

        except Exception:
            pass

        # UI overlay
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, str(counter), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Exception in thread Thread-370 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_19100\3500496605.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-371 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
   

Rep: 1
Rep: 2


Exception in thread Thread-379 (_speak):
Traceback (most recent call last):
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\renad\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\renad\AppData\Local\Temp\ipykernel_19100\3500496605.py", line 16, in _speak
  File "c:\Users\renad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
